In [1]:
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
label_list = []
img_list = []

In [3]:
path = 'hairremoved'
all_images = os.listdir(path)
i = 0
for ix in tqdm(os.listdir(path)):
    out = path +"/" + all_images[i]
    img = cv2.imread(out)
    img = cv2.resize(img, (224,224))
    img_list.append(img)
    label_list.append(1)
    i = i + 1

100%|██████████| 4120/4120 [00:20<00:00, 204.47it/s]


In [4]:
path = 'nonmel_hairremoved'
all_images = os.listdir(path)
i = 0
for ix in tqdm(os.listdir(path)):
    if i < 4121:
        out = path +"/" + all_images[i]
        img = cv2.imread(out)
        if img is None:
            print('Wrong path:', path)
        else:
            img = cv2.resize(img, (224,224))
            img_list.append(img)
            label_list.append(0)
    i = i + 1

100%|██████████| 6154/6154 [00:19<00:00, 314.95it/s]


In [6]:
img = np.array(img_list)
lbl = np.array(label_list)

In [7]:
img.shape

(8241, 240, 240, 3)

In [8]:
X_train, X_test,y_train, y_test = train_test_split(img,lbl, test_size=0.1,random_state=32, shuffle= True) 

In [16]:
from tensorflow.keras.applications.mobilenet import MobileNet, decode_predictions, preprocess_input
vgg_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

C:\Users\Ayberk\anaconda3\envs\tfp\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [17]:
for layer in vgg_model.layers:
    layer.trainable = True

In [18]:
modelVGG16 = Sequential()
modelVGG16.add(vgg_model)
modelVGG16.add(Flatten())
modelVGG16.add(Dense(1024, activation='relu'))
modelVGG16.add(Dropout(0.2))
modelVGG16.add(Dense(512, activation='relu'))
modelVGG16.add(Dropout(0.2))
modelVGG16.add(Dense(256, activation='relu'))
modelVGG16.add(Dropout(0.2))
modelVGG16.add(Dense(128, activation='relu'))
modelVGG16.add(Dropout(0.2))
modelVGG16.add(Dense(64, activation='relu'))
modelVGG16.add(Dropout(0.2))
modelVGG16.add(Dense(32, activation='relu'))
modelVGG16.add(Dropout(0.2))
modelVGG16.add(Dense(1, activation='sigmoid'))

# Compile model 
modelVGG16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                          tf.keras.metrics.Precision(),
                                                                          tf.keras.metrics.Recall()])

In [19]:
modelVGG16.fit(X_train, y_train,batch_size=10, epochs=5)

Train on 7416 samples
Epoch 1/5
7416/7416 [==============================] - 116s 16ms/sample - loss: 0.9297 - accuracy: 0.6458 - precision_1: 0.6558 - recall_1: 0.6197
Epoch 2/5
7416/7416 [==============================] - 111s 15ms/sample - loss: 0.6180 - accuracy: 0.7001 - precision_1: 0.7042 - recall_1: 0.6943
Epoch 3/5
7416/7416 [==============================] - 112s 15ms/sample - loss: 0.5949 - accuracy: 0.7074 - precision_1: 0.7054 - recall_1: 0.7164
Epoch 4/5
7416/7416 [==============================] - 112s 15ms/sample - loss: 0.5634 - accuracy: 0.7408 - precision_1: 0.7615 - recall_1: 0.7043
Epoch 5/5
7416/7416 [==============================] - 112s 15ms/sample - loss: 0.5489 - accuracy: 0.7364 - precision_1: 0.7523 - recall_1: 0.7080


In [20]:
modelVGG16.evaluate(X_test, y_test, verbose=1)

825/825 [==============================] - 3s 4ms/sample - loss: 0.4968 - accuracy: 0.7503 - precision_1: 0.8577 - recall_1: 0.5768


[0.496832092964288, 0.75030303, 0.8576779, 0.5768262]